Visa matrix to list of Available countries

In [22]:
import pandas as pd


# Choose a specific country
COUNTRY_TARGET = 'Russia'

# Assuming you have a DataFrame 'visa_data' with the visa information
visa_data = pd.read_csv('../data/passport-index-matrix.csv', index_col=0)

# Function to get visa information in the specified format
def get_visa_info(country, category, description):
    return f"{country}, {category}, {description}"

# Categories that are considered valid
available_categories = ['visa free', 'visa on arrival', 'e-visa'] + list(map(str, range(10, 361)))

# Function to get visa information in the specified format for a specific country
def get_visa_info_for_country(country_target, visa_data):
    visa_info = []
    for country in visa_data.columns:
        if country_target != country:
            category = visa_data.loc[country_target, country]
            if category in available_categories:
                description = "Country description"  # Replace with actual description
                visa_info.append(get_visa_info(country, category, description))
    return visa_info

# Get and print visa information for the chosen country
visa_info_for_target_country = get_visa_info_for_country(COUNTRY_TARGET, visa_data)

# Write visa information to a CSV file
output_file_path = '../data/available_countries.csv'
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write("countries,category,description\n")
    for info in visa_info_for_target_country:
        file.write(info + '\n')
    


Available countries to KML file with borders

In [7]:
from bs4 import BeautifulSoup
import csv

def read_kml(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        soup = BeautifulSoup(content, 'xml')
        placemarks = soup.find_all('Placemark')
        countries = {}
        for placemark in placemarks:
            name = placemark.find('name').text.strip()
            geometry = str(placemark.find('MultiGeometry')) 
            countries[name] = {'geometry': geometry}
        return countries

def process_csv(csv_file, countries_data):
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        header = next(reader)
        rows = list(reader)

    output_data = []

    for row in rows:
        country_name = row[0]
        if country_name in countries_data:
            category = row[1]
            description = row[2]
            geometry = countries_data[country_name]['geometry']
            
            output_data.append({
                'category': category,
                'name': country_name,
                'description': description,
                'geometry': geometry
            })
        else:
            print(f"Country not found: {country_name}")

    return header, output_data

def write_kml(output_file, header, data):
    with open(output_file, 'w', encoding='utf-8') as out_file:
        out_file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        out_file.write('<kml xmlns="http://www.opengis.net/kml/2.2">\n')
        out_file.write('<Document>\n')

        for entry in data:
            out_file.write(f'  <Placemark>\n')
            out_file.write(f'    <name>{entry["category"]} дней</name>\n')
            out_file.write(f'    <description>{entry["description"]}</description>\n')
            out_file.write(f'    {entry["geometry"]}\n')
            out_file.write(f'  </Placemark>\n')

        out_file.write('</Document>\n')
        out_file.write('</kml>\n')

# Read from KML
countries_data = read_kml('../data/countries.kml')

header, output_data = process_csv('../data/in.csv', countries_data)

# Write to KML
write_kml('out.kml', header, output_data)


Country not found: Macau
Country not found: Eswatini
Country not found: Brunei Darussalam
Country not found: Hong Kong
